## Import Libraries and Set Up the Environment


In [143]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense


## Importing Dataset

In [144]:
dataset = pd.read_csv('medical_data.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [145]:
X

array([['Constant fatigue and muscle weakness, struggling to stay awake.',
        'Chronic Fatigue Syndrome'],
       ['Frequent severe migraines, sensitivity to light and sound.',
        'Migraine with Aura'],
       ['Sudden weight gain and feeling cold, especially in the hands and feet.',
        'Hypothyroidism'],
       ['High fever, sore throat, and swollen lymph nodes, feeling very weak.',
        'Mononucleosis'],
       ['Excessive thirst and frequent urination, dry mouth persistently.',
        'Diabetes Mellitus'],
       ['Sharp, stabbing chest pain that worsens when lying down or inhaling deeply.',
        'Pericarditis'],
       ['Unexplained weight loss and night sweats, feeling feverish.',
        'Tuberculosis'],
       ['Severe abdominal pain radiating to the back, nausea without vomiting.',
        'Pancreatitis'],
       ['Joint swelling and stiffness in the morning, especially in the fingers and wrists.',
        'Rheumatoid Arthritis'],
       ['Recurrent, painf

## Spliting data into Training and Test Sets

In [146]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [147]:
X_train

array([['Hair loss in patches.', 'Alopecia Areata'],
       ['Persistent sore throat and fever.', 'Strep Throat'],
       ['Frequent urination and a burning sensation.',
        'Urinary Tract Infection'],
       ['Frequent migraines that last for hours.', 'Migraine'],
       ["I've been getting nosebleeds more frequently than usual.",
        'Epistaxis'],
       ['Irregular heartbeat and feeling jittery.',
        'Atrial Fibrillation'],
       ['Recurrent sore throat.', 'Pharyngitis'],
       ['Noticed a lump on my neck that has been growing.',
        'Thyroid Cancer'],
       ['A persistent dry cough that has lasted for more than a month.',
        'Chronic Bronchitis'],
       ['Experiencing sharp chest pains occasionally, especially under stress.',
        'Angina Pectoris'],
       ['A sudden increase in frequency and intensity of headaches.',
        'Migraines'],
       ['Loss of appetite and metallic taste in the mouth.',
        'Gastroesophageal Reflux Disease'],
       ['

## Label Encoding

In [151]:
label = LabelEncoder()
X_train[:, 0] = label.fit_transform(X_train[:, 0])
X_train[:, 1] = label.fit_transform(X_train[:, 1])

In [152]:
X_train

array([[103, 4],
       [237, 135],
       [100, 151],
       [95, 102],
       [159, 54],
       [182, 17],
       [240, 122],
       [215, 141],
       [3, 29],
       [76, 9],
       [4, 104],
       [187, 67],
       [0, 29],
       [248, 27],
       [82, 10],
       [41, 18],
       [110, 123],
       [186, 130],
       [47, 56],
       [77, 81],
       [122, 30],
       [142, 14],
       [134, 89],
       [157, 30],
       [87, 30],
       [251, 143],
       [305, 117],
       [173, 35],
       [160, 84],
       [243, 64],
       [312, 118],
       [246, 104],
       [258, 131],
       [93, 114],
       [10, 48],
       [196, 119],
       [14, 82],
       [79, 30],
       [7, 110],
       [199, 125],
       [70, 102],
       [227, 17],
       [136, 80],
       [64, 97],
       [161, 85],
       [259, 77],
       [148, 103],
       [54, 127],
       [133, 143],
       [19, 88],
       [74, 26],
       [192, 22],
       [113, 113],
       [236, 135],
       [30, 44],
       [315, 8